In [1]:
import enviroments_package
from enviroments_package import RemoveKeyObservationWrapper, ScaleRewardWrapper, ScaleActionWrapper
import gymnasium

from stable_baselines3.common.env_util import SubprocVecEnv
from stable_baselines3.common.vec_env import VecMonitor
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import configure
from sb3_contrib import RecurrentPPO

import shutil
import os
import threading
from datetime import datetime
import time

In [9]:
#se crea una función para fabricar cada entorno con los wrapers correspondientes para un SubprocVecEnv
def make_env():
    def _init():
        # Crea el entorno base
        env = gymnasium.make('drone_tfg_juanes/Drone-v1', simulation_path=world_dir, reward_json_path=json_reward, no_render=False)

        # Aplica los wrappers necesarios
        env = RemoveKeyObservationWrapper(env, remove_keys=["camera", "gps"])
        env = ScaleRewardWrapper(env, scale_factor=0.1)
        env = ScaleActionWrapper(env, in_low=-1, in_high=1, out_low=0, out_high=576)
        return env
    return _init

#creamos un callback para que el entorno funcione correctamente y no haya problemas con el entorno
class TrainingCallback(BaseCallback):
    def __init__(self, env, verbose=1):
        super(TrainingCallback, self).__init__(verbose)
        self.env = env

    def _on_step(self) -> bool:
        return True

    def _on_rollout_start(self) -> None:
        self.env.reset()

    def _on_training_end(self):
        print("Entrenamiento finalizado. Cerrando el entorno...")
        self.env.close()

#aquí creamos un archivo para mover el progress.csv que geenra cada entrenamiento a la carpeta de data_collected
def move_and_rename_csv(src_dir, dst_dir, new_name):
    # Definir el archivo CSV específico a buscar
    csv_file = 'progress.csv'
    src_path = os.path.join(src_dir, csv_file)

    # Verificar si el archivo 'progress.csv' existe en el directorio de origen
    if not os.path.exists(src_path):
        print("No se encontró el archivo 'progress.csv' en el directorio de origen.")
        return

    # Definir la ruta de destino con el nuevo nombre
    dst_path = os.path.join(dst_dir, new_name)

    # Mover y renombrar el archivo
    shutil.copy2(src_path, dst_path)
    print(f"Archivo copiado y renombrado a {dst_path}")


#creo una función para modificar el learning rate y el entropy coefficient
def schedule_rate(initial_value, final_value, total_cycles, current_cycle):
    return final_value if current_cycle >= total_cycles else initial_value + (final_value - initial_value) * (current_cycle / total_cycles)

In [3]:
import os
import pandas as pd
from datetime import datetime
from stable_baselines3.common.evaluation import evaluate_policy

def update_model(model, env, log_dir='./logs', n_eval_episodes=10):
    """
    Evalúa el rendimiento del modelo y actualiza un archivo de evaluación en CSV.

    Args:
        model: Modelo de Stable-Baselines3 a evaluar.
        env: Entorno de entrenamiento.
        log_dir (str): Carpeta para almacenar el archivo de evaluación.
        n_eval_episodes (int): Número de episodios para evaluar el modelo.

    Returns:
        bool: True si el modelo supera la recompensa máxima registrada, False en caso contrario.
    """
    eval_file_path = os.path.join(log_dir, "evaluate.csv")

    # Crear el directorio y archivo si no existen
    os.makedirs(log_dir, exist_ok=True)
    if not os.path.exists(eval_file_path):
        # Crear archivo vacío con encabezado
        pd.DataFrame(columns=["reward", "timestamp"]).to_csv(eval_file_path, index=False)

    # Leer el archivo de evaluación
    eval_df = pd.read_csv(eval_file_path)

    # Comprobar si el archivo tiene datos
    if not eval_df.empty:
        max_reward = eval_df["reward"].max()
    else:
        max_reward = float('-inf')

    # Evaluar el modelo en el entorno dado
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=n_eval_episodes, return_episode_rewards=False)

    # Comparar la recompensa y actualizar el archivo si es necesario
    if mean_reward > max_reward:
        # Registrar la nueva recompensa y timestamp en el archivo CSV
        new_row = pd.DataFrame({
            "reward": [mean_reward],
            "timestamp": [datetime.now().strftime("%Y%m%d_%H%M%S")]
        })
        new_row.to_csv(eval_file_path, mode='a', header=False, index=False)
        result = True
    else:
        result = False
    env.close()
    return result

In [4]:
#aquí están los archivos del simulador y la configuración de la recompensa
world_dir = "/Users/jeste/Desktop/Clase/TFG/drone_tfg_juanes/simulation_package/worlds/my_frst_webots_world.wbt"
json_reward = "/Users/jeste/Desktop/Clase/TFG/drone_tfg_juanes/configs/reward_package_config/motors_use.json"
# Define el número de entornos que se van a crear
num_envs = 4

#direcciones de dónde se guardará cada componente iportante del modelo
model_dir = "./models/ppomodel"
log_dir = "./logs/"
data_collected_dir = './data_collected/'
os.makedirs(log_dir, exist_ok=True)

#los valores de learning rate y entropy coefficient
lr = 1e-3
ent_coef = 0.1

#define los pasos totales que se usarán para entrenar al modelo en cada ciclo
timesteps = 20480
#define los pasos que se usrán antes de actualizar los pesos del modelo
n_steps = 1024
#define los paquetes de experiencia que se usarán para actualizar los pesos
batch_size = 64

#ciclos
n_cycles = 2
timeout_threshold = 240

#creamos el entorno
env = SubprocVecEnv([make_env() for _ in range(num_envs)])
env = VecMonitor(env)#, filename=f'./data_collected/ppo_monitor{datetime.now().strftime("%Y%m%d_%H%M%S")}')

In [5]:
new_logger = configure(log_dir, ["stdout", "csv"])
callback = TrainingCallback(env=env, verbose=1)

if not os.path.exists(model_dir+".zip"):
    print("first train")
    model = RecurrentPPO(
        "MultiInputLstmPolicy",
        env,
        verbose=1,                    # Si quiero ver las acciones por terminal
        n_steps=n_steps,              # Controla el buffer de experiencias para actualizar la política
        batch_size=batch_size,        # Tamaño del lote, separa el buffer de experiencias en paquetes de este tamaño
        learning_rate=lr,     # Tasa de aprendizaje
        ent_coef=ent_coef     # Coeficiente de entropía para exploración
    )
    model.set_logger(new_logger)
    model.learn(total_timesteps=timesteps, callback=callback)
    model.save(model_dir)

    move_and_rename_csv(log_dir, data_collected_dir, f'ppo_data{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv')
else:
    print("retrainning")
    model = RecurrentPPO.load(model_dir+".zip", env=env)
    model.set_logger(new_logger)

    model.learning_rate = lr
    model.ent_coef = ent_coef

    model.learn(total_timesteps=timesteps, callback=callback)
    time.sleep(5)
    move_and_rename_csv(log_dir, data_collected_dir, f'ppo_data{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv')

    if update_model(model, make_env()(), n_eval_episodes=10):
        model.save(path=model_dir)



Logging to ./logs/
retrainning
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.9     |
|    ep_rew_mean     | 6.8      |
| time/              |          |
|    fps             | 10       |
|    iterations      | 1        |
|    time_elapsed    | 392      |
|    total_timesteps | 4096     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 14.6      |
|    ep_rew_mean          | 7.76      |
| time/                   |           |
|    fps                  | 9         |
|    iterations           | 2         |
|    time_elapsed         | 842       |
|    total_timesteps      | 8192      |
| train/                  |           |
|    approx_kl            | 0.0262491 |
|    clip_fraction        | 0.252     |
|    clip_range           | 0.2       |
|    entropy_loss         | -7.32     |
|    explained_variance   | 0.586     |
|    learning_rate        | 0

C:\Users\jeste\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


WEBOTS Result: INFO: xyz_controller: Starting controller: python.exe -u xyz_controller.py
Simulation Starting
RESET 
SET_ALL_MOTORS [227.20082 180.52658 101.46528 115.27259]
SET_ALL_MOTORS [221.20131  175.14601   93.88118  107.961685]
SET_ALL_MOTORS [215.64673 171.32141  88.88364 103.93745]
SET_ALL_MOTORS [210.87537 168.29091  85.09205 101.29978]
SET_ALL_MOTORS [206.68198 165.75739  82.01155  99.37503]
SET_ALL_MOTORS [202.9137   163.56879   79.3921    97.862076]
SET_ALL_MOTORS [199.48552  161.6471    77.10277   96.627174]
SET_ALL_MOTORS [196.36661  159.96825   75.14143   95.703705]
SET_ALL_MOTORS [193.4288  158.41472  73.24729  94.72788]
SET_ALL_MOTORS [190.70859 157.03717  71.56137  93.9563 ]
SET_ALL_MOTORS [188.16335 155.79091  70.01961  93.29279]
SET_ALL_MOTORS [185.77582 154.65753  68.59556  92.71522]
SET_ALL_MOTORS [183.52757 153.62389  67.27878  92.21719]
RESET 
SET_ALL_MOTORS [227.20082 180.52658 101.4653  115.27259]
SET_ALL_MOTORS [221.16655  175.11267   93.796326 107.85057 ]
S

In [6]:
init = make_env()
show_env = init()

model = RecurrentPPO.load("models/ppomodel", env=env)

observation, _ = show_env.reset()

for i in range(100):
    action, _states = model.predict(observation, deterministic=True)
    observation, reward, terminated, truncated, _ = show_env.step(action)

    if terminated:
        observation, _ = show_env.reset()

show_env.close()

KeyboardInterrupt: 